# Bradley-Terryモデル（を雑に拡張して）Mリーガーの強さを推定する

**このノートブックでやること**：1対1で行うゲームのプレイヤの強さを推定するために使われるBradley-Terryモデルを、麻雀用に拡張して、Mリーガーの強さを対局結果（NOT打牌内容）から統計的に推測します。

<span style="color:red">**注意**</span>：このノートブックで示されるMリーガーの強さは、対局の結果・スコアから（雑に拡張した）**Bradley-Terryモデルで推定**されたものです。
この記事で示された強さは、あくまで推定したものであり、また、モデルが異なれば結果も変わる、ということにご注意ください（つまり、このノートブックで示される強さが唯一絶対の正しい強さの値だ！**ということではありません**）。

## Bradley-Terryモデル

### 問題設定
1対1で行うゲームの、$M$人のプレイヤが行った、$N$回の対戦結果の集合があるとします。
対戦結果としては、「どちらが勝ったか（or 負けたか）」が与えられます。
この$N$回の対戦結果の集合の中には、同じ組合せが複数存在しても良いです。
この対戦結果の集合を用いて、$M$人のプレイヤの強さを推定したいです。

### 定式化
Bradley-Terry（BT）モデルでは、プレイヤ$i \in [M]$がプレイヤ$j \in [M]$に勝つ確率は以下で定義されます：
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = \sigma(s_i - s_j),
$$
ここで、$\mathbf{s} \in \mathbb{R}^{M}$は推定・学習されるパラメータ、$\sigma: \mathbb{R} \to (0, 1)$はロジスティックシグモイド関数
$$
    \sigma(z) = \frac{1}{1 + \exp (-z)}
$$
です。
直観的には、$s_i \in \mathbb{R}$は$i$番目のプレイヤの強さを表しています：BTモデルでは$s_i - s_j$をロジスティックシグモイド関数に通すので、
 - $s_i - s_j$が大きければ大きいほどプレイヤ$i$がプレイヤ$j$に勝つ確率が高く、
 - $s_i - s_j$が小さければ小さいほどプレイヤ$i$がプレイヤ$j$に勝つ確率が低く、
 - $s_i = s_j$のとき、プレイヤ$i$がプレイヤ$j$に勝つ確率は$50$%と
 
推定されます。
BTモデルの気持ちが良い点として、プレイヤ$i$の勝率 = 1 - プレイヤ$j$の勝率が成り立つ、すなわち、
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = 1 - f_{\mathrm{BT}}(j \text{ beats } i; \mathbf{s})
$$
が成り立つことが挙げられます（=2人のプレイヤの勝率について矛盾が生じない。例えば、$i$と$j$が対戦について、$i$の勝率は90%、$j$の勝率は80%といったような勝率予測はされない）。

### BTモデルの推定
$M$人のプレイヤの間の、$N$回の対戦結果の集合
$$
    \mathcal{D} = \{ (i_n, j_n, y_n)\}_{n=1}^{N} \in \{[M] \times [M] \times \{0, 1\}\}^N
$$
が与えられたとします。ここで、$i_n, j_n \in [M]$は、$n \in [N]$ 番目の対戦を行ったプレイヤ、$y_n \in \{0, 1\}$は対戦結果を表します：$y_n=1$は$i_n$の勝ちを、$y_n=0$は$j_n$の勝ちを意味します。
$\mathcal{D}$が与えられたときのBTモデルの推定方法は様々考えられますが、例えば最尤推定を行う場合、$y_n$が$f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s})$を母数として持つベルヌーイ分布にしたがうと仮定して、以下の負の対数尤度を最小化すれば良いです：
$$
    -\sum_{n=1}^N \left[ y_n \log f_{\mathrm{BT}}(i_n \text{ beats } j_n; \mathbf{s}) + (1-y_n) \log f_{\mathrm{BT}}(j_n \text{ beats } i_n; \mathbf{s})\right].
$$
また、（$\mathbf{s}$の事前分布としてガウス分布を仮定した場合の）MAP推定では、上の式にさらに$\alpha \lVert \mathbf{s} \rVert_2^2 / 2$を加えたものを最小化すれば良いです（$\alpha > 0$は正則化項の強さを表すハイパーパラメータ）。

最尤推定・MAP推定の最適化問題はどちらも解析的には解けませんが、平滑で凸な関数の最小化問題であるため、勾配ベースの方法で効率よく解くことができます。

### BTモデルのロジスティック回帰としての表現
写像$x: [M] \times [M] \to \{0, 1, -1\}^M$を次のように定義します：
$$
    x(i, j)_k = \begin{cases}
    1 & \text{if } k=i,\\
    -1 & \text{if } k =j,\\
    0 & \text{otherwise}.
    \end{cases}
$$
このとき、BTモデルは以下のように書くことができます：
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = \sigma(s_i - s_j) = \sigma(\langle x(i, j), \mathbf{s}\rangle).
$$
ここで、$\langle \cdot , \cdot \rangle$は通常の内積です（２つのベクトルの要素ごとの積の和）。
すなわち、プレイヤ$i,j$に対するBTモデルは、$x(i,j) \in \{0, 1, -1\}^M$に対するロジスティック回帰と解釈できます。
したがって、BTモデルを推定したい場合、$\mathcal{D}$から$\mathcal{D}' = \{(x(i_n, j_n), y_n\}_{n=1}^N$を作り、$\mathcal{D}'$上でロジスティック回帰を学習すれば良いです（例えば`sklearn`を使う）。

### BTモデルの良いところ
プレイヤの強さの指標として、まず「勝率」を考える人も多いと思います。
すべてのプレイヤの組について、十分な数の対戦が行われていれば、勝率は良い指標となるでしょう。
しかし、実世界の問題においては、全てのプレイヤの組について十分な数の対戦が行われていない場合が多いでしょう。
また、より悪いことに、対戦が行われていないプレイヤの組が多数存在することも珍しくないはずです。
このような場合、各プレイヤの勝率は対戦相手の偏りに影響を受けます：熟練度の低いプレイヤとの対戦割合が多いプレイヤは強さが過大評価され、熟練度の高いプレイヤとの対戦割合が多いプレイヤは強さが過剰評価されてしまいます。
一方で、BTモデルでは、自身と対戦相手の強さ（と解釈ができる・に対応する）パラメータに応じて勝率が計算されます。
そのため、対戦回数が十分でなく、各プレイヤの対戦相手の熟練度・強さに偏りがある場合であっても、BTモデルはプレイヤの強さを上手く推定することができます。

### BTモデルの限界
BTモデルでは、各プレイヤは「強さ」に対応するスカラのみを持ちます。
強さに対応するパラメータの値が大きいプレイヤは勝ちやすく、小さいプレイヤは負けやすい、と推定されます。
したがって、BTモデルでは「プレイヤの相性」といった要素を考えることができません。
たとえば、「グーしか出さないプレイヤ1」「チョキしか出さないプレイヤ2」「パーしか出さないプレイヤ3」の3人にじゃんけんを行ってもらうとします。
このとき、十分な数の対戦が行われたとしても、「プレイヤ1はプレイヤ2に有利」「プレイヤ1はプレイヤ3に不利」といったことは推定できず、単に「全員同じくらいの強さ」と推定されます。
「相性」を考慮したい場合は、BTモデルではなくBlade-Chestモデルを用いると良いです。


## BTR<sup>4</sup>モデル：BTモデルの麻雀のための拡張
BTモデルは以下のようなゲームに使うことができるのでした：
 - 1対1で行い、
 - 対戦結果としては勝ち負けだけが与えられる。
 
一方で、麻雀は以下のような特徴を持ちます：
 - 4人で行い、
 - 対戦結果として「勝ち負け」だけでなく「点数」が与えられる、4人の点数の合計は0点である。
 
したがって、麻雀に対してBTモデルをそのまま使うことはできません。

そこで、BTモデルを拡張して麻雀に使えるようにします。
以下では、まず、対戦結果として勝ち負けだけでなく点数（スコア）が与えられる場合に拡張します。
そして次に、4人で行う場合に拡張します。

### 麻雀のための拡張1：対戦結果として点数が与えられる場合
対戦結果として、勝ち負けだけでなく、二人のプレイヤのスコアが与えられるとします。
ただし、二人のプレイヤの点数の合計は$0$であるとします。
前述の通り、BTモデルは、プレイヤ$i$がプレイヤ$j$に勝つ確率を、$\sigma(s_i - s_j)$と定義しており、また、$s_i, s_j$はそれぞれプレイヤ$i$と$j$の強さ（として解釈ができるの）でした。
そこで、BTモデルをこの問題設定に拡張したモデル、Bradley-Terry Regression（BTR）モデル$f_{\mathrm{BTR}}(\cdot, \cdot; \mathbf{s}): [M]^2 \to \mathbb{R}$を以下のように定義します：
$$
  f_{\mathrm{BTR}}(i, j; \mathbf{s}) = s_i - s_j.
$$
ここで、 $f_{\mathrm{BTR}}(i, j; \mathbf{s})$は、プレイヤ$i$とプレイヤ$j$が対戦したときのプレイヤ$i$のスコアです。 
当然、BTRモデルにおける、プレイヤ$i$とプレイヤ$j$が対戦したときのプレイヤ$j$のスコアは
$$
  f_{\mathrm{BTR}}(j, i; \mathbf{s}) =  s_j - s_i
$$
であり、この２つの点数の合計は明らかに$0$です（気持ち良い！）。

BTRモデルの推定は、例えば最尤推定を行う場合、「プレイヤ$i_n$とプレイヤ$j_n$が対戦した場合のプレイヤ$i_n$の点数$y_n\in \mathbb{R}$」は「$s_{i_n} - s_{j_n}$を平均とするガウス分布」にしたがうと仮定して、以下の負の対数尤度

$$
  \frac{1}{2} \sum_{n=1}^N \left(f_{\mathrm{BTR}}(i_n, j_n; \mathbf{s}) - y_n\right)_2^2 = \frac{1}{2} \sum_{n=1}^N \left(\left(s_{i_n} - s_{j_n}\right) - y_n\right)_2^2
$$
を最小化する等が考えられます（ガウス分布でなく他の分布でも良い、最適化できるかは分からないが）。
また、$s_i - s_j = \langle x(i, j), \mathbf{s}\rangle$であるので、前述のBTモデルがロジスティック回帰として解釈できるのと同様に、BTRモデルは線形回帰と解釈できます。

### 麻雀のための拡張2：4人で行う場合
プレイヤが4人である場合、対戦結果は$[M]^4 \times \mathbb{R}^4$の要素として表現されます。
この問題設定に対しては、BTRモデルを2入力1出力から4入力4出力に拡張、すなわち、$[M]^2 \to \mathbb{R}$から$[M]^4 \to \mathbb{R}^4$に拡張すれば良さそうです（スコアの和が0であるという制約の関係で、$[M]^4 \to \mathbb{R}^3$で良いですが、簡単のため、$[M]^4 \to \mathbb{R}^4$とします）。
もとのBTRモデルに対して、4出力に拡張されたBTRモデルをBTR<sup>4</sup>モデルと呼ぶことにします。

前述のBTRモデルでは、「自身の強さ -　相手の強さ」として、自身のスコアが定義されていました。
そこで、BTR<sup>4</sup>モデルでは、ある対戦における自分のスコアを、「（自身の強さ - 相手1の強さ） + （自身の強さ - 相手2の強さ） + （自身の強さ - 相手3の強さ）」と定義することにします。
フォーマルに書くと、BTR<sup>4</sup>モデルでは、プレイヤ$i_1, i_2, i_3, i_4\in [M]$が対戦した場合のプレイヤ$i_j$、$j \in [4]$のスコアを以下のように定義します：
$$
  f_{\mathrm{BTR}}^4(i_1, \ldots, i_4; \mathbf{s})_j = \sum_{k=1}^4 (s_j - s_k).
$$
例えば、プレイヤ1,2,4,6が対戦したときの、プレイヤ4のスコアは
$$
  f_{\mathrm{BTR}}^4({1,2,4,6}; \mathbf{s})_3 = (s_4 - s_1) + (s_4 - s_2) + (s_4 - s_6)
$$
となります。
この定義は対戦を4人で行う場合のものですが、より一般に、$K$人で行う場合に拡張すると以下のようになります：
$$
    f_{\mathrm{BTR}}^K(i_1, \ldots, i_K; \mathbf{s})_j = \sum_{k=1}^{K} (s_{i_j} - s_{i_k}).
$$
BTR<sup>2</sup>モデルはBTRモデルと一致するため、この意味で、BTR<sup>K</sup>モデルはBTRモデルの拡張となっています。

今、対戦結果の集合として、
$$
    \mathcal{D} = \{(i_{n,1}, i_{n,2}, i_{n,3}, i_{n,4}, y_{n, 1}, y_{n,2}, y_{n,3}, y_{n,4})\}_{n=1}^N
$$
が与えられたとします。
ここで、$i_{n, j}$は、$n \in [N]$番目の対戦結果における$j \in [4]$番目のプレイヤであり、$y_{n, j}$はそのマッチにおける$j$番目のプレイヤのスコアです。
このとき、BTR<sup>4</sup>モデルの推定は、例えば最尤推定を行う場合、以下の負の対数尤度
$$
  \frac{1}{2} \sum_{n=1}^N \sum_{j=1}^4 \left(f_{\mathrm{BTR}}^4(i_{n, 1}, \ldots, i_{n,4})_j- y_{n, j}\right)^2 = \frac{1}{2} \sum_{n=1}^N \sum_{j=1}^4 \left( \sum_{k=1}^4 \left(s_{n, i_j} - s_{n, i_k}\right) - y_{n, j}\right)^2
$$
を最小化すれば良いです。

また、写像$x': [M]^4 \to \{0, 1, -1\}^M$を次のように定義します：
$$
    x'(i_1, \{i_2, i_3, i_4\})_k = \begin{cases}
    3 & \text{if } k=i_1,\\
    -1 & \text{if } k \in \{i_2, i_3, i_4\},\\
    0 & \text{otherwise}.
    \end{cases}
$$
このとき、$f_{\mathrm{BTR}}^4(i_1, \ldots, i_4; \mathbf{s})_j = \sum_{k=1}^{4} (s_{i_j} - s_{i_k}) = \langle x'(i_j, \{i_1, \ldots, i_4\} \setminus \{i_j\}), \mathbf{s}\rangle$となります。
したがって、$N$個のデータ上でのBTR<sup>4</sup>モデルの学習は、（$4N$個の観測データ上の）線型回帰の学習とみなせます。

以上がBTモデルの麻雀のための拡張の大まかな説明になります。

### この拡張の気持ち悪い点
ある一回の対戦結果$(i_1, i_2, i_3, i_4, y_{i_1}, y_{i_2}, y_{i_3}, y_{i_4})$が与えられたとき、$y_{i_1}, y_{i_2}, y_{i_3}, y_{i_4}$はそれぞれ独立に観測されている、と暗に仮定しています（線型回帰との関係より）。
実際はそうではないでしょうから、この点は気持ち悪い・雑に感じます。

## 実際に推定する
長々と説明しましたが、実際に、Mリーグの2021/4/19までの対局結果のデータを用いて、Mリーガーの強さを推定してみます。

### データの読み込み

In [1]:
import numpy as np
from load_dataset import remove_ranking_point, add_ranking_point, load_mleague_dataset
from bradley_terry import BT
from blade_chest import BladeChestInner
from sklearn.model_selection import GridSearchCV

X, y = load_mleague_dataset("./data/")
y_original = remove_ranking_point(y) # 素点

print(X)
print(y)
print(y_original)
n_players = len(np.unique(X))
print(f"プレイヤ数：{n_players}")

[['瑞原明奈' '魚谷侑未' '日向藍子' '内川幸太郎']
 ['沢崎誠' '滝沢和典' '瀬戸熊直樹' '多井隆晴']
 ['滝沢和典' '多井隆晴' '園田賢' '佐々木寿人']
 ...
 ['瀬戸熊直樹' '松本吉弘' '勝又健志' '前原雄大']
 ['白鳥翔' '内川幸太郎' '小林剛' '瀬戸熊直樹']
 ['朝倉康心' '前原雄大' '多井隆晴' '村上淳']]
[[ 77.2  10.1 -26.  -61.3]
 [ 62.9  20.9 -21.8 -62. ]
 [ 61.9   5.8 -23.6 -44.1]
 ...
 [ 61.2   7.6 -18.2 -50.6]
 [ 60.4   3.4 -20.  -43.8]
 [ 56.   13.9 -23.1 -46.8]]
[[ 32.2   5.1 -11.  -26.3]
 [ 17.9  15.9  -6.8 -27. ]
 [ 16.9   0.8  -8.6  -9.1]
 ...
 [ 16.2   2.6  -3.2 -15.6]
 [ 15.4  -1.6  -5.   -8.8]
 [ 11.    8.9  -8.1 -11.8]]
プレイヤ数：30


### BTR<sup>4</sup>モデルの学習 - 順位点も込み

In [2]:
bt = BT(alpha=1e-6) # alpha：正則化項の強さ
bt.fit(X, y)
strength_sorted = np.sort(bt.coef_)[::-1]
players_sorted = bt.enc_.inverse_transform(np.argsort(bt.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

強さランキング
['堀慎吾' '黒沢咲' '多井隆晴' '村上淳' '小林剛' '佐々木寿人' '近藤誠一' '日向藍子' '内川幸太郎' '鈴木たろう'
 '魚谷侑未' '滝沢和典' '白鳥翔' '沢崎誠' '松本吉弘' '勝又健志' '園田賢' '茅森早香' '朝倉康心' '瀬戸熊直樹'
 '石橋伸洋' '二階堂亜樹' '岡田紗佳' '前原雄大' '瑞原明奈' '高宮まり' '藤崎智' '萩原聖人' '和久津晶' '丸山奏子']
強さ
[ 4.67805308  2.62451839  2.58078302  2.21608517  1.92121629  1.71645411
  1.64857158  1.33297904  1.25921974  1.02275803  0.85122006  0.48996463
  0.08956458 -0.01922415 -0.13781155 -0.19652878 -0.50648242 -0.6528878
 -0.70901174 -0.80259377 -0.97524326 -1.0443925  -1.26574682 -1.27227157
 -1.45766153 -1.52075303 -1.85641586 -2.90674021 -3.41186452 -3.69575821]


上の結果からわかるように、「堀慎吾プロが一番強い」という推定結果になりました。
とくに、2番目に強いと推定された黒沢咲プロと堀慎吾プロの強さの差はおよそ2.05ポイントで、この値は「順位が並んでいるプレイヤ同士の強さの差」で一番大きく（その次に大きいのは、藤崎智プロと萩原聖人プロの差で、およそ1.05ポイント）、堀慎吾プロが頭一つ抜けて強いと推定されているように見えます。

### BTR<sup>4</sup>モデルの学習 - 順位点抜き
BTR<sup>4</sup>モデルでは、プレイヤのスコアは「"自身の強さ - 相手の強さ"の和」として定義されています。
そのため、直観的には、順位点をそのまま使うのはやや不自然に思えます。
そこで、今度は素点を用いてプレイヤの強さを推定してみます。

In [3]:
bt = BT(alpha=1e-6)
bt.fit(X, y_original)
strength_sorted = np.sort(bt.coef_)[::-1]
players_sorted = bt.enc_.inverse_transform(np.argsort(bt.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

強さランキング
['堀慎吾' '多井隆晴' '小林剛' '黒沢咲' '佐々木寿人' '村上淳' '滝沢和典' '近藤誠一' '内川幸太郎' '魚谷侑未'
 '鈴木たろう' '勝又健志' '沢崎誠' '松本吉弘' '瀬戸熊直樹' '朝倉康心' '茅森早香' '園田賢' '高宮まり' '日向藍子'
 '白鳥翔' '二階堂亜樹' '岡田紗佳' '石橋伸洋' '瑞原明奈' '前原雄大' '藤崎智' '萩原聖人' '丸山奏子' '和久津晶']
強さ
[ 1.4726661   1.0682637   1.052745    1.00116057  0.76588415  0.69553108
  0.57507113  0.52406937  0.33470813  0.30920012  0.29058544  0.24950908
  0.22716601  0.14107229 -0.1066164  -0.11048185 -0.18462958 -0.23928799
 -0.26941475 -0.28029142 -0.2843175  -0.30631124 -0.48360523 -0.54697546
 -0.63728646 -0.70479965 -0.7150433  -0.91635503 -1.12388546 -1.79833087]


この場合でも、「堀慎吾プロが一番強い」という結果になりました。
2位以下は結果がいくらか変わっています。
例えば、多井隆晴プロや小林剛プロ、佐々木寿人プロが順位を上げています。
一方で、日向藍子プロは大きく順位を落としています。
実際、[「Mリーグ成績速報（非公式）」様が集計・公開されている2020年度レギュラーシーズンまでのデータ](https://twitter.com/mleague_results/status/1376890867768729602/photo/1)を見てみると、日向藍子プロは素点はマイナスですが順位点では大きくプラスになっており、この結果の違いはそれなりに妥当そうに思えます。


## 今回のオチ

さて、この強さの推定結果は妥当なのでしょうか？
推定結果の妥当性の検証方法はいくらかありますが、ここではテストデータに対する予測誤差を計算し検証します：データの全てではなく8割（てきとう）を使ってBTR<sup>4</sup>モデルを推定し、その推定結果をもとに残りの2割の対戦結果のスコアを予測し、実際の試合結果とどの程度ズレがあるかを計算します。
予測と実際の結果のズレは、平均二乗誤差で評価します：
$$
  \frac{1}{2 \cdot 4 \cdot N_{\mathrm{test}}} \sum_{n=1}^{N_{\mathrm{test}}} \sum_{j=1}^4(\hat{y}_{n, j} - y_{n, j})^2.
$$

ここで、$y_{n, j}$は$n$番目の対戦結果に対する$j$番目のプレイヤの実際のスコアで、$\hat{y}_{n, j}$はその予測です。
平均二乗誤差は「予測スコアと実際のスコアのズレ」を表しているので、低ければ低いほどよいです。

BTR<sup>4</sup>モデルの推定結果の良さを測るために、「全ての試合で全てのプレイヤのスコアを0と推定した場合」、すなわち、「なにもわからない」と推定した場合の予測のズレも計算し、BTR<sup>4</sup>モデルを用いて予測した場合と比較します。
BTR<sup>4</sup>モデルが上手くプレイヤの強さを推定できていれば、BTR<sup>4</sup>モデルの平均二乗誤差は、「すべて0」=「なにもわからない」と推定した場合と比較してずっとずっと低い値になります。

In [4]:
n_train = len(X) * 8 // 10
bt = BT(alpha=1e-10)

bt.fit(X[:n_train], y[:n_train])
y_pred = bt.predict(X[n_train:])
print(f"テスト2乗誤差:{0.5*np.mean((y[n_train:]-y_pred)**2)}")
print(f"全て0と推定したときのテスト2乗誤差：{0.5*np.mean(y[n_train:]**2)}")

bt.fit(X[:n_train], y_original[:n_train])
y_pred = bt.predict(X[n_train:])
print(f"テスト2乗誤差:{0.5*np.mean((y_original[n_train:]-y_pred)**2)}")
print(f"全て0と推定したときのテスト2乗誤差：{0.5*np.mean(y_original[n_train:]**2)}")

テスト2乗誤差:1025.511295041669
全て0と推定したときのテスト2乗誤差：982.6600438596491
テスト2乗誤差:136.72694102486253
全て0と推定したときのテスト2乗誤差：132.69513157894735


**BTR<sup>4</sup>モデルに基づいて予測するよりも「すべて0」=「なにもわからない」と予測したほうが良い！！解散！！**